In [59]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

import time

In [60]:
country = "AR"
nq_date = "190912"

In [61]:
census = pd.read_csv("../../../Data/"+country+"/sample/"+"ipumsi_00015.csv")

In [99]:
netquest = pd.read_csv("../../../Data/"+country+"/panel/"+country+"_netquest-panel.csv")

In [63]:
nq_dict = pd.read_excel("../../../Data/AR/panel/AR_levels.xlsx",
                       )

In [64]:
geo1_nq = "AR_provincia"
geo2_nq = "AR_departamento"
geo1_ipums = 'GEO1_AR2010'
geo2_ipums = 'GEO2_AR2010'
year = '2010'

In [65]:
ipums_geo2 = pd.read_csv('../../../Data/'+country+'/sample/ipums_codebook_'+geo2_ipums+'.csv',encoding='Latin1', names=['code','name'], skiprows=1)
ipums_geo1 = pd.read_csv('../../../Data/'+country+'/sample/ipums_codebook_'+geo1_ipums+'.csv',encoding='Latin1', names=['code','name'], skiprows=1)

Make sure Marcos Paz is here:

In [66]:
ipums_geo2[ipums_geo2.name.str.contains('Marcos')]

,code,name
64,6050,Marcos Paz
132,14010,Marcos Juárez


In [67]:
nq_geo1 = nq_dict[nq_dict.Variable==geo1_nq]
nq_geo1.columns = ["Variable",geo1_nq,geo1_nq+"_name"]
nq_geo1 = nq_geo1[[geo1_nq,geo1_nq+"_name"]]

nq_geo2 = nq_dict[nq_dict.Variable==geo2_nq]
nq_geo2.columns = ["Variable",geo2_nq,geo2_nq+"_name"]
nq_geo2 = nq_geo2[[geo2_nq,geo2_nq+"_name"]]
# nq_geo1 = pd.read_excel("../Data/AR/panel/netquest_codebook_AR_provincia.xlsx")

# nq_geo2 = pd.read_excel("../Data/AR/panel/netquest_codebook_AR_departamento.xlsx")

Setting up the unique DF for geographies for IPUMS

In [68]:
ipums_geodf = census[[geo1_ipums,geo2_ipums]]
ipums_geodf.columns = ['geo1_code','geo2_code']
ipums_geodf.drop_duplicates(subset=['geo1_code','geo2_code'],inplace=True)

ipums_geodf['geo1_name'] = ipums_geodf.merge(ipums_geo1, 
                                             how='left', 
                                             left_on = "geo1_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

ipums_geodf['geo2_name'] = ipums_geodf.merge(ipums_geo2, 
                                             how='left', 
                                             left_on = "geo2_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

We have to get the melting action going here.

In [69]:
ipums_geodf = ipums_geodf.geo2_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo2_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo2_name')], value_name = "geo2_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo2_name'])
ipums_geodf.geo2_name = ipums_geodf.geo2_name.str.strip()

In [70]:
ipums_geodf = ipums_geodf.geo1_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo1_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo1_name')], value_name = "geo1_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo1_name'])
ipums_geodf.geo1_name = ipums_geodf.geo1_name.str.strip()

Accounting for the CABA mess. Reducing to just one centroid for all of CABA.

In [71]:
ipums_geodf.loc[ipums_geodf.geo1_name=="City of Buenos Aires",
             'geo1_name'] = 'CABA'
ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA"),'geo2_name'] = 'CABA'

In [72]:
caba_codes = ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code

Dropping CABA-related duplicates:

In [73]:
ipums_geodf.drop_duplicates(subset=["geo1_name","geo2_name"], inplace=True)

In [74]:
ipums_geodf.shape

(511, 4)

In [75]:
ipums_geodf[ipums_geodf.geo1_name=="CABA"]

,geo1_code,geo2_code,geo2_name,geo1_name
0,2,2006,CABA,CABA


Make sure all CABAs in the census table have that same leftover geo2_code:

In [76]:
ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code.values[0]

2006

In [77]:
census.loc[census[geo2_ipums].isin(caba_codes),geo2_ipums] = \
len(census.loc[census[geo2_ipums].isin(caba_codes),geo2_ipums])*[ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code[0]]

#= ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code.values

Now to Netquest--first, get the geo1- and geo2_name in there.

Import barrio as departamento in CABA--actually, that turns out not to be a great idea, since it seems that these are actually partidos of the province.

Instead, give these a unique code, that will be used to identify them later.

In [78]:
# netquest.loc[netquest[geo2_nq].isna()&netquest.AR_barrio.notna(),geo2_nq
#             ] = netquest.AR_barrio[netquest[geo2_nq].isna()&netquest.AR_barrio.notna()]

In [101]:
netquest.loc[netquest[geo2_nq].isna()&(netquest[geo1_nq]==1),geo2_nq] = 999

In [81]:
# province_labels = pd.read_excel("../Data/CL/provincia_dict.xlsx", encoding='Latin1', header=None)
nq_geodf = netquest.merge(nq_geo2, on=geo2_nq, how='left')\
                   .merge(nq_geo1, on=geo1_nq, how='left')[[geo1_nq,geo1_nq+'_name',geo2_nq,geo2_nq+'_name']]
nq_geodf.columns = ['geo1_code','geo1_name','geo2_code','geo2_name']
nq_geodf.drop_duplicates(inplace=True)

# Fixing CABA not having Dept.
nq_geodf.loc[nq_geodf.geo1_name=="Ciudad Autónoma de Buenos Aires",
             'geo1_name'] = 'CABA'
# Where province is CABA and no departamento exists, call these "CABA"
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isna()),'geo2_name'] = 'CABA'

nq_geodf = nq_geodf[(nq_geodf['geo1_code'].notna()) & (nq_geodf['geo2_code'].notna())] 

In [82]:
nq_geodf[nq_geodf.geo1_name=="CABA"]
nq_geodf[nq_geodf.geo1_code==1]

,geo1_code,geo1_name,geo2_code,geo2_name
6,1.0,CABA,999.0,NaN
18539,1.0,CABA,133.0,Villarino
33152,1.0,CABA,39.0,Escobar
66903,1.0,CABA,21.0,Cañuelas
87680,1.0,CABA,2.0,9 de Julio
94989,1.0,CABA,67.0,La Matanza
106522,1.0,CABA,8.0,Avellaneda
110353,1.0,CABA,128.0,Tres Arroyos
117681,1.0,CABA,156.0,Sobremonte


In [83]:
nq_geodf[nq_geodf.duplicated('geo2_code', keep=False)].sort_values('geo2_code')

,geo1_code,geo1_name,geo2_code,geo2_name
297,2.0,Buenos Aires,2.0,9 de Julio
87680,1.0,CABA,2.0,9 de Julio
55,2.0,Buenos Aires,8.0,Avellaneda
106522,1.0,CABA,8.0,Avellaneda
40,2.0,Buenos Aires,21.0,Cañuelas
66903,1.0,CABA,21.0,Cañuelas
33,2.0,Buenos Aires,39.0,Escobar
33152,1.0,CABA,39.0,Escobar
8,2.0,Buenos Aires,67.0,La Matanza
94989,1.0,CABA,67.0,La Matanza


Finding the duplicates in nq_geodf.geo2_code, and keeping the combinations that occur most often (which are quite clearly the good ones).

In [84]:
nq_geodf['count'] = nq_geodf.apply(
    lambda r: sum((netquest[geo1_nq]==r['geo1_code'])&
                  (netquest[geo2_nq]==r['geo2_code'])),
                                   axis=1)
# nq_geodf.sort_values(['geo2_code','count']) \
#     [(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep=False))]
nq_geodf = nq_geodf.sort_values(['geo2_code','count']) \
    [~(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep='last'))]

In [85]:
nq_geodf.shape

(504, 5)

In [86]:
ipums_geodf[ipums_geodf.geo1_name.str.contains("CABA")]

,geo1_code,geo2_code,geo2_name,geo1_name
0,2,2006,CABA,CABA


In [87]:
nq_geodf[nq_geodf.geo1_name.str.contains("CABA")]

,geo1_code,geo1_name,geo2_code,geo2_name,count
6,1.0,CABA,999.0,NaN,17031


Next, a function for fuzzy-joining columns from dataframes

In [88]:
def fuzzy_join(df1, df2, varname, subset=False, cutoff=90):
    '''Returns a dataframe the length of df1, with the matches on the given variable
    
        Assumes the var is named the same in both dataframes.
        
        subset is a tuple or list of length 2 whose first element is a string indicating which
        variable in df1 has to be equal to which variable in df2 (specified by 2nd element)
        '''
    #figure out if there is a constraint
    if not subset:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)
        
    else:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)    

    
    # different match cases
    #morRatioMatch = ratio_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    oneRatioMatch = ratio_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noRatioMatch = ratio_matches.apply(lambda l: (l[0][1]<cutoff))

    #morPartiMatch = parti_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    onePartiMatch = parti_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noPartiMatch = parti_matches.apply(lambda l: (l[0][1]<cutoff))
    
    # pick out what's better
    matches = pd.Series([(np.nan,np.nan,np.nan)]*len(df1),index=df1.index)
    matches.loc[oneRatioMatch] = ratio_matches.loc[oneRatioMatch].apply(lambda l: l[0])
    matches.loc[(noRatioMatch&onePartiMatch)] = parti_matches.loc[(noRatioMatch&onePartiMatch)]\
        .apply(lambda l: l[0])
    matches.loc[(noRatioMatch&noPartiMatch)] = parti_matches.loc[(noRatioMatch&noPartiMatch)]\
        .apply(lambda l: l[0])
    
    df = pd.DataFrame()
    df['name'] = matches.apply(lambda l: l[0])
    df['score'] = matches.apply(lambda l: l[1])
    df['index'] = matches.apply(lambda l: l[2])
    df['parti_matches'] = parti_matches
    df['ratio_matches'] = ratio_matches
    
    return df

Use the new tool!

Manual fixes, as found necessary in the next step...

In [102]:
nq_geodf.loc[nq_geodf.geo2_code==999, 'geo2_name'] = "CABA"

ipums_geodf.loc[ipums_geodf.geo1_name.str.contains("Buenos Aires province"),
             'geo1_name'] = "Buenos Aires"

ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Puan"),
                'geo2_name'] = "Puán"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("General San Martín"))\
                &(ipums_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "Ciudad Libertador San Martín"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("La Capital"))\
                &(ipums_geodf.geo1_name=="San Luis"),
                'geo2_name'] = "Juan Martín de Pueyrredón"
ipums_geodf.loc[(ipums_geodf.geo2_name=="Maipú")\
                &(ipums_geodf.geo2_code==6050),
                'geo2_name'] = "Marcos Paz"



ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Chascomus")),
                'geo2_name'] = "Chascomús"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Jose C. Paz")),
                'geo2_name'] = "José C. Paz"
# nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Adolfo Gonzales Chaves")),
#                 'geo2_name'] = "Adolfo González Chaves"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Paso de Indios"))\
                &(nq_geodf.geo1_name=="Chubut"),
                'geo2_name'] = "Paso de los Indios"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Coronel de Marina Leonardo Rosales"))\
                &(nq_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "Coronel de Marine L. Rosales"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Veinticinco de Mayo"))\
                &(ipums_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "25 de Mayo"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Pueyrredón"))\
                &(nq_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "General Pueyrredón"

We see that a lot of depa's are listed under various provinces. Can we utilize subpar matching later to get just the ones that are good?

The CABA districts require special attention. They are labelled school districts, but they are actually _comunas_.

The Barrios aren't provided for much of Netquest, so I'm just simplifying to the whole city here.

In [103]:

# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XIII')),
#              'geo2_name'] = 'Comuna 13'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('VIII')),
#              'geo2_name'] = 'Comuna 8'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XIV')),
#              'geo2_name'] = 'Comuna 14'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XII')),
#              'geo2_name'] = 'Comuna 12'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('VII')),
#              'geo2_name'] = 'Comuna 7'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('III')),
#              'geo2_name'] = 'Comuna 3'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XV')),
#              'geo2_name'] = 'Comuna 15'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XI')),
#              'geo2_name'] = 'Comuna 11'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('IX')),
#              'geo2_name'] = 'Comuna 9'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('VI')),
#              'geo2_name'] = 'Comuna 6'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('IV')),
#              'geo2_name'] = 'Comuna 4'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('II')),
#              'geo2_name'] = 'Comuna 2'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('X')),
#              'geo2_name'] = 'Comuna 10'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('V')),
#              'geo2_name'] = 'Comuna 5'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('I')),
#              'geo2_name'] = 'Comuna 1'

In [104]:
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Puerto Madero','San Nicolás','Retiro','San Telmo','Constitución'])),
#             'geo2_name'] = 'Comuna 1'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Recoleta'])),
#             'geo2_name'] = 'Comuna 2'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Balvanera','San Cristóbal'])),
#             'geo2_name'] = 'Comuna 3'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['La Boca','Barracas', 'Parque Patricios', 'Nueva Pompeya'])),
#             'geo2_name'] = 'Comuna 4'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Almagro','Boedo'])),
#             'geo2_name'] = 'Comuna 5'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Caballito'])),
#             'geo2_name'] = 'Comuna 6'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Flores','Parque Chacabuco'])),
#             'geo2_name'] = 'Comuna 7'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Soldati', 'Villa Lugano', 'Villa Riachuelo'])),
#             'geo2_name'] = 'Comuna 8'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Parque Avellaneda', 'Mataderos', 'Liniers'])),
#             'geo2_name'] = 'Comuna 10'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Luro', 'Vélez Sársfield', 'Floresta', 'Monte Castro', 'Villa Real', 'Versalles'])),
#             'geo2_name'] = 'Comuna 10'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Devoto', 'Villa del Parque', 'Villa Santa Rita', 'Villa General Mitre'])),
#             'geo2_name'] = 'Comuna 11'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Pueyrredón', 'Villa Urquiza', 'Coghlan', 'Saavedra'])),
#             'geo2_name'] = 'Comuna 12'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Núñez', 'Belgrano', 'Colegiales'])),
#             'geo2_name'] = 'Comuna 13'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Palermo'])),
#             'geo2_name'] = 'Comuna 14'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Ortúzar', 'Chacarita', 'Villa Crespo', 'La Paternal', 'Agronomía', 'Parque Chas'])),
#             'geo2_name'] = 'Comuna 15'

## Fuzzy joining

In [105]:
nq_geodf[['geo1_match_name','geo1_match_score','geo1_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo1_name')[['name','score','index']]

In [106]:
nq_geodf[['geo2_match_name','geo2_match_score','geo2_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo2_name', 
             subset=['geo1_match_name','geo1_name']
            )[['name','score','index']]

## De-duping

Checking how good this worked.

In [107]:
# dupes = nq_geodf[nq_geodf.duplicated('geo2_code')].geo2_code

tmp = nq_geodf.copy()

# for code in dupes: 
#     if 'CABA' in tmp.loc[tmp.geo2_code==code].geo1_match_name.unique():
#         if int(tmp.loc[((tmp.geo2_code==code) & (tmp.geo1_name=="Buenos Aires")), 'geo2_match_score'])==100:
#             tmp = tmp[~((tmp.geo2_code==code) & (tmp.geo1_name=="CABA"))]
#     # if one of the geo2_match_names is an exact match, throw out any others
#     if sum(tmp.loc[tmp.geo2_code==code,"geo2_name"]==tmp.loc[tmp.geo2_code==code,"geo2_match_name"])>0:
#         tmp = tmp[~((tmp.geo2_code==code) & (tmp.geo2_name!=tmp.geo2_match_name))]

**What is still an issue??**

In [108]:
tmp[tmp.duplicated('geo2_code', keep=False)].sort_values('geo2_code')\
[["geo1_code","geo2_code",
  "geo1_name","geo1_match_name",
  "geo2_name","geo2_match_name",
  "count"]]

,geo1_code,geo2_code,geo1_name,geo1_match_name,geo2_name,geo2_match_name,count


Success! This can be the geo-df.

In [109]:
nq_geodf = tmp

In [110]:
nq_geodf[(nq_geodf.geo2_match_score<71)][['geo1_name',
                                          'geo1_code',
#                                         'geo1_match_name',
                                        'geo2_name',
                                        'geo2_code',
                                        'geo2_match_name',
                                        'geo2_match_score'
                                       ]
                                      ].sort_values('geo1_name'
                                      )#.to_csv("./faulty_matches.csv")

,geo1_name,geo1_code,geo2_name,geo2_code,geo2_match_name,geo2_match_score


In [111]:
ipums_geodf[["geo1_name","geo2_name","geo2_code"]
           ][ipums_geodf.geo2_name.str.contains("Marcos Paz")]

,geo1_name,geo2_name,geo2_code
80,Buenos Aires,Marcos Paz,6050


In [112]:
has_ipums_geo = nq_geodf.geo2_match_score>70 #&nq_geodf.geo1_match_index.notna()

In [113]:
sum(has_ipums_geo)
len(nq_geodf)

504

In [114]:
nq_geodf['IPUMS_geo2_code'] = np.nan

nq_geodf.loc[has_ipums_geo,'IPUMS_geo2_code'] = nq_geodf[has_ipums_geo]\
                            .geo2_match_index\
                            .astype(int)\
                            .apply(
                                lambda i: ipums_geodf.loc[i,'geo2_code']
                            .astype(int)
)

We want to replace each nq_geo2 with the _code_ for the corresponding census geography.



Read in the municipio-centroids, and hopefully attach them here.

In [115]:
geo2_centroids = pd.read_csv('../../../Data/'+country+'/geography/'+country+'_geo2_centroids.csv', encoding='latin1')

Merge centroids onto NQ geometries:

In [116]:
nq_geodf_merged = nq_geodf.merge(geo2_centroids[['ADMIN_NAME','Y','X','IPUM'+year]], 
               left_on='IPUMS_geo2_code',
               right_on="IPUM"+year,
               how='left'
              ).drop('IPUM'+year, axis=1)

In [117]:
nq_geodf['IPUMS_geo2_code'].unique()
# nq_geodf.geo2_code.unique()

array([ 6058,  6055,  6004,  6081,  6095,  6072,  6018,  6047,  6027,
        6040,  6063,  6071,  6014,  6032,  6048,  6082,  6005,  6033,
        6049,  6085,  6002,  6009,  6080,  6092,  6054,  6059,  6065,
        6101,  6003,  6062,  6053,  6052,  6083,  6006,  6078,  6066,
        6039,  6016,  6073,  6100,  6079,  6068,  6077,  6074,  6090,
        6051,  6026,  6045,  6028,  6061,  6069,  6086,  6012,  6013,
        6022,  6017,  6031,  6099,  6043,  6093,  6010,  6097,  6036,
        6060,  6001,  6050,  6064,  6096,  6089,  6094,  6011,  6034,
        6037,  6076,  6084,  6035,  6044,  6075,  6029,  6056,  6098,
        6088,  6008,  6091,  6070,  6087,  6007,  6024,  6015,  6021,
        6042,  6057,  6030,  6041,  6019,  6046,  6067,  6023,  6025,
        6038, 14018, 14007, 14005, 14015, 14013, 14002, 14019, 14016,
       14010, 14021, 14003, 14006, 14020, 14012, 14009, 14014, 14017,
       14004, 14011, 14001, 14008, 82007, 82010, 82008, 82003, 82012,
       82017, 82005,

Merge NQ geometries onto NQ data:

In [118]:
panel_geo = netquest.merge(nq_geodf_merged[['X','Y','geo2_code']],
               left_on=geo2_nq,
               right_on='geo2_code',
               how='left'
              )

In [119]:
panel_geo.shape[0]==netquest.shape[0]

True

In [120]:
panel_geo.to_csv('../../../Data/'+country+'/panel/'+country+'_netquest-panel_geo.csv')

And the same for census: attach geographies

In [121]:
census_geo = census.merge(geo2_centroids[['ADMIN_NAME','X',"Y",'IPUM'+year]],
                          left_on = geo2_ipums,
                          right_on='IPUM'+year,
                          how='left'
                         ).drop('IPUM'+year,axis=1)

In [122]:
census_geo.shape[0]==census.shape[0]

True

In [123]:
caba_codes

0     2006
1     2015
2     2008
3     2004
4     2011
5     2012
6     2003
7     2009
8     2014
9     2013
10    2007
11    2005
12    2001
13    2002
14    2010
Name: geo2_code, dtype: int64

In [124]:
census[census.GEO2_AR2010.isin(caba_codes)]

,COUNTRY,YEAR,SAMPLE,SERIAL,PERSONS,HHWT,GEO1_AR,GEO1_AR2010,GEO2_AR,GEO2_AR2010,...,AGE,SEX,EMPSTAT,EMPSTATD,AR2010A_EDLEV,AR2010A_WKLOOK,EMPSTAT_HEAD,EMPSTATD_HEAD,AR2010A_EDLEV_HEAD,AR2010A_WKLOOK_HEAD
0,32,2010,32201001,1000,2,10,32002,2,32002001,2006,...,65,1,1,110,7,9,NaN,NaN,NaN,NaN
1,32,2010,32201001,1000,2,10,32002,2,32002001,2006,...,62,2,3,300,6,2,NaN,NaN,NaN,NaN
2,32,2010,32201001,2000,1,10,32002,2,32002001,2006,...,52,1,1,110,7,9,NaN,NaN,NaN,NaN
3,32,2010,32201001,3000,1,10,32002,2,32002001,2006,...,55,1,1,110,2,9,NaN,NaN,NaN,NaN
4,32,2010,32201001,4000,1,10,32002,2,32002001,2006,...,17,2,1,110,4,9,NaN,NaN,NaN,NaN
5,32,2010,32201001,5000,1,10,32002,2,32002001,2006,...,20,1,1,110,4,9,NaN,NaN,NaN,NaN
6,32,2010,32201001,6000,3,10,32002,2,32002001,2006,...,20,1,1,110,2,9,NaN,NaN,NaN,NaN
7,32,2010,32201001,6000,3,10,32002,2,32002001,2006,...,17,2,1,120,3,9,NaN,NaN,NaN,NaN
8,32,2010,32201001,6000,3,10,32002,2,32002001,2006,...,0,1,0,0,1,9,NaN,NaN,NaN,NaN
9,32,2010,32201001,7000,2,10,32002,2,32002001,2006,...,50,2,1,110,2,9,NaN,NaN,NaN,NaN


In [125]:
census_geo.to_csv('../../../Data/'+country+'/sample/'+country+'_ipums-census_geo.csv')